# 3.1. Diseño de la base de datos
Se desea implementar una serie de funciones para contruir nuestra BD clave-valor que imite el funcionamiento de Twitter.
Funciones:
- nuevo_usuario
- nuevo_follower
- nuevo_following
- seguir
- nuevo_post

## Configuración

En primer lugar vamos a necesitar gestionar las propiedades de conexión. Para conectarnos a REDIS necesitamos 

In [ ]:
import configparser

# Wrapper del configparser de python.
class RedisConfig(object):
    
    def __init__(self, config_file=None):
        self._config = configparser.ConfigParser()
        if config_file is not None :
            self._config.read(config_file)
    
    @property
    def host(self):
        return self._config['redis']['REDIS_HOST']
    @property
    def port(self):
        return self._config['redis']['REDIS_PORT']
    @property
    def password(self):
        return self._config['redis']['REDIS_PWD']
    @property
    def db(self):
        return self._config['redis']['REDIS_DB']
    

In [2]:
# Sustituir propiedades por las del propias de cada uno.
# En mi caso son localhost, 6379, '' y 1.
! test -f redis_config.ini && rm redis_config.ini && echo 'Se ha borrado redis_config.ini'
! touch redis_config.ini
! echo '[redis]' >> redis_config.ini
! echo 'REDIS_HOST=localhost' >> redis_config.ini
! echo 'REDIS_PORT=6379' >> redis_config.ini
! echo 'REDIS_PWD=' >> redis_config.ini
! echo 'REDIS_DB=1' >> redis_config.ini
! echo
! cat redis_config.ini
! echo
! echo 'redis_config.ini creado en: '$(pwd)

Se ha borrado redis_config.ini

[redis]
REDIS_HOST=localhost
REDIS_PORT=6379
REDIS_PWD=
REDIS_DB=5

redis_config.ini creado en: /Users/adzarei/Documents/UNED/ING_CIENC_DATOS/projects/GAI/redis/src/notebooks


In [3]:
config = RedisConfig('redis_config.ini') 
print(config.host)
print(config.port)
print(config.password)
print(config.db)

localhost
6379

5


## Modelo
Ahora vamos a diseñar los objetos que necesitamos para el dominio. Necesitaremos:
- Usuario
- Post

## Redis API  \#1
Es hora de implementar los métodos que usaremos para la carga de datos:
- nuevo_usuario(username)
- nuevo_follower(username, follower, timestamp)
- nuevo_following(username, followee, timestamp)
- seguir(username, followee, timestamp)
- nuevo_post(username, body, timestamp)

In [4]:
import redis
r = redis.Redis(host=config.host, port=config.port, password=config.password, db=config.db)

In [5]:
def nuevo_usuario(username):
    if r.hget('users', username) is None:
        id = r.incr('user_id')
        r.hmset('users', {id : username, username:id})

In [6]:
def _nuevo_follower(username, follower, timestamp):
    
    usr_id = r.hget('users', username)
    follower_id = r.hget('users', follower)
    
    if usr_id is None or follower_id is None:
        return False
    
    usr_id = usr_id.decode('utf-8')
    follower_id = follower_id.decode('utf-8')
    
    r.zadd('ts_follower_of:' + usr_id, {timestamp : follower_id})
    r.sadd('follower_of:' + usr_id, follower_id)
    
    return True

In [7]:
def _nuevo_following(username, followee, timestamp):
    usr_id = r.hget('users', username)
    followee_id = r.hget('users', followee)
    
    if usr_id is None or followee_id is None:
        return
    
    usr_id = usr_id.decode('utf-8')
    followee_id = followee_id.decode('utf-8')
    
    r.zadd('ts_followee_of:' + usr_id, {timestamp : followee_id})

In [8]:
def seguir(username, followee, timestamp):
    res = _nuevo_follower(followee, username, timestamp)
    # Si el _nuevo_follower devuelve falso, no se ejecuta _nuevo_following.
    res = res and _nuevo_following(username, followee, timestamp)
    return res

In [9]:
def nuevo_post(username, body, timestamp):
    usr_id = r.hget('users', username)
    
    if usr_id is None:
        return
    
    usr_id = usr_id.decode('utf-8')
    
    post_id = r.incr('post_id')
    followers = r.smembers('follower_of:' + usr_id)
    
    # Creamos el post
    r.hmset('post:' + str(post_id), {'id' : post_id, 'timestamp' : timestamp, 'body' : body})
    
    # Guardamos la referencia para el usuario
    r.zadd('ts_timeline:' + usr_id, {timestamp : post_id})
    
    # Guardamos la referencia para los followees del usuario.
    for follower in followers:
        follower_id = follower.decode("utf-8")
        r.zadd('ts_timeline:' + follower_id, {timestamp : post_id})
        

    

In [10]:
nuevo_usuario('zap')
nuevo_usuario('Maria')
nuevo_usuario('patri')
nuevo_usuario('conhic')

In [11]:
r.hget('users', 'zap').decode("utf-8")

'1'

In [12]:
seguir('Maria', 'zap','ahora')
seguir('zap', 'patri' ,'ahora')
seguir('patri','conhic' ,'ahora')
seguir('conhic','Maria' ,'ahora')
seguir('conhic','zap' ,'ahora')
seguir('patri','zap' ,'ahora')

In [13]:
r.smembers('follower_of:1')

{b'2', b'3', b'4'}

In [14]:
nuevo_post('zap', 'test esto es un test', 'luego') # 1
nuevo_post('zap', 'test esto es un test', 'luego1') # 2
nuevo_post('zap', 'test esto es un test', 'luego2') # 3
nuevo_post('zap', 'test esto es un test', 'luego3') # 4
nuevo_post('patri', 'test esto es un test', 'luego4') # 5

In [15]:
# zap
r.zscan('ts_timeline:1', 0)

(0,
 [(b'luego', 1.0),
  (b'luego1', 2.0),
  (b'luego2', 3.0),
  (b'luego3', 4.0),
  (b'luego4', 5.0)])

In [16]:
# Maria
r.zscan('ts_timeline:2', 0)

(0, [(b'luego', 1.0), (b'luego1', 2.0), (b'luego2', 3.0), (b'luego3', 4.0)])

In [17]:
# Patri
r.zscan('ts_timeline:3', 0)

(0,
 [(b'luego', 1.0),
  (b'luego1', 2.0),
  (b'luego2', 3.0),
  (b'luego3', 4.0),
  (b'luego4', 5.0)])

In [18]:
# Conhic
r.zscan('ts_timeline:4', 0)

(0, [(b'luego', 1.0), (b'luego1', 2.0), (b'luego2', 3.0), (b'luego3', 4.0)])

In [19]:
r.hgetall('post:5')

{b'id': b'5', b'timestamp': b'luego4', b'body': b'test esto es un test'}